# 1. Algoritmo de la división de polinomios evitando fracciones.

1. Tomamos mcm de denominadores de ambos polinomios (divisor y dividendo) en caso de que sean fracciones y así quedan u' y v' con coeficientes enteros.
2. Además preparamos la pseudo-division calculando grados de ambos polinomios quitando ceros de grados altos.
3. Implementamos pseudo-división calculando el alpha como vimos en clase, allí usamos también poli_por_const definida antes.
4. Definimos una función extra para simplificar las fracciones
5. La última función usa todas las demás, obteniendo los mcm de denominadores, **definiendo** u prima y v prima, llamando la pseudo-división, luego desnormalizando, simplificando y retornando cociente, residuo y el alppha que se usó en pseudo-división

In [1]:
from fractions import Fraction
from math import gcd
from functools import reduce
from typing import List, Tuple


def mcm(a: int, b: int) -> int:
    return abs(a * b) // gcd(a, b)

def mcm_list(lst: List[int]) -> int:
    return reduce(mcm, lst, 1)  # Reduce aplica la función mcm acumulativamente empezando desde 1

def mcm_denominadores(fracciones: List[Fraction]) -> int:
    denominadores = [f.denominator for f in fracciones if f != 0]
    if not denominadores:
        return 1
    return mcm_list(denominadores)

def grado(p: List[int]) -> int:
    for i in reversed(range(len(p))):
        if p[i] != 0:
            return i
    return -1

def quitar_ceros_derecha(p: List[int]) -> List[int]:
    d = grado(p)
    return p[:d + 1] if d >= 0 else [0]

def poli_por_const(p: List[int], c: int) -> List[int]:
    return [coeff * c for coeff in p]

def pseudo_division(u: List[int], v: List[int]) -> Tuple[List[int], List[int], int]:
    u = quitar_ceros_derecha(u)      
    v = quitar_ceros_derecha(v)     

    m = grado(u)                     
    n = grado(v)                     
    delta = m - n                   
    vn = v[n]                       
    alpha = int(pow(vn, delta + 1)) 

    r = poli_por_const(u, alpha)    
    q = [0] * (delta + 1)           

    for i in range(delta, -1, -1):
        ri = r[n + i]               
        if ri == 0:                 
            q[i] = 0
            continue
        q[i] = ri // vn             
        for j in range(n + 1):      
            r[i + j] -= q[i] * v[j]

    return quitar_ceros_derecha(q), quitar_ceros_derecha(r), alpha  

def simplificar_fracciones(p: List[Fraction]) -> List[Fraction]:
    denominadores = [f.denominator for f in p if f != 0]
    if not denominadores:
        return [Fraction(0)]
    mcm_den = mcm_list(denominadores)
    numeradores = [f.numerator * (mcm_den // f.denominator) for f in p]
    mcd_num = reduce(gcd, [abs(n) for n in numeradores if n != 0], 0) or 1
    return [Fraction(n // mcd_num, mcm_den) for n in numeradores]

def pseudo_division_racional(u: List[Fraction], v: List[Fraction]) -> Tuple[List[Fraction], List[Fraction], Fraction]:
    gamma = mcm_denominadores(u)       
    
    beta = mcm_denominadores(v)        

    u_prim = [int(c * gamma) for c in u]  
    v_prim = [int(c * beta) for c in v]   
    q_int, r_int, alpha = pseudo_division(u_prim, v_prim)  

    factor_q = Fraction(gamma, beta * alpha)  
    factor_r = Fraction(1, beta * alpha)     

    q_real = [Fraction(c) * factor_q for c in q_int]  
    r_real = [Fraction(c) * factor_r for c in r_int]  

    q_real = simplificar_fracciones(q_real) 
    r_real = simplificar_fracciones(r_real) 

    return q_real, r_real, Fraction(alpha)   


In [2]:
#ejemplo del libro:
u = [Fraction(3), Fraction(1), Fraction(1), Fraction(2)]
v = [Fraction(1), Fraction(-2), Fraction(3)]

q, r, alpha = pseudo_division_racional(u, v)

print("Cociente exacto q(x):", q)
print("Residuo exacto r(x):", r)
print("Factor alpha utilizado:", alpha)

Cociente exacto q(x): [Fraction(7, 9), Fraction(2, 3)]
Residuo exacto r(x): [Fraction(20, 9), Fraction(17, 9)]
Factor alpha utilizado: 9


In [3]:
# u(x) = (3/2)x^3 - (4/3)x^2 + 2  → [2, 0, -4/3, 3/2]
# v(x) = (1/5)x^2 - (2/3)x + 1   → [1, -2/3, 1/5]

u = [Fraction(2), Fraction(0), Fraction(-4, 3), Fraction(3, 2)]
v = [Fraction(1), Fraction(-2, 3), Fraction(1, 5)]

q, r, alpha = pseudo_division_racional(u, v)

print("Cociente exacto q(x):", q)
print("Residuo exacto r(x):", r)
print("Factor alpha utilizado:", alpha)

Cociente exacto q(x): [Fraction(22, 15), Fraction(3, 5)]
Residuo exacto r(x): [Fraction(-98, 15), Fraction(17, 9)]
Factor alpha utilizado: 9


In [4]:
# u(x) = (6)x^3 + (7/15)x^2 - x + 2  → [2, -1, 7/15, 6]
# v(x) = (3/2)x^2 - (21)x + 12   → [12, -21, 3/2]

u = [Fraction(2), Fraction(-1), Fraction(7, 15), Fraction(6)]
v = [Fraction(12), Fraction(-21), Fraction(3, 2)]

q, r, alpha = pseudo_division_racional(u, v)

print("Cociente exacto q(x):", q)
print("Residuo exacto r(x):", r)
print("Factor alpha utilizado:", alpha)

Cociente exacto q(x): [Fraction(1267, 2), Fraction(45, 1)]
Residuo exacto r(x): [Fraction(-5053, 1), Fraction(17003, 2)]
Factor alpha utilizado: 9


# 2. Máximo común divisor para polinomios.


In [1]:
def mcd_primitivo(p, q):
    """
    Algoritmo Euclidiano Primitivo para polinomios con coeficientes enteros.
    Entrada: polinomios p y q
    Salida: mcd(p, q)
    """

    def contenido(polinomio):
        """Contenido: MCD de los coeficientes del polinomio"""
        if polinomio == 0:
            return 0
        coeficientes = [c for c in polinomio.coefficients() if c != 0]
        return gcd(coeficientes) if coeficientes else 0

    def parte_primitiva(polinomio):
        """Parte primitiva = polinomio / contenido"""
        cont = contenido(polinomio)
        return polinomio // cont if cont != 0 else polinomio

    def pseudoresto(u, v):
        """Pseudoresto de u dividido por v"""
        if v == 0 or u.degree() < v.degree():
            return u
        lc = v.leading_coefficient()
        d = u.degree() - v.degree() + 1
        u_mod = (lc^d) * u
        _, resto = u_mod.quo_rem(v)
        return resto

    # Paso 1: calcular el contenido común
    c = gcd(contenido(p), contenido(q))

    # Paso 2: reducir a partes primitivas
    u = parte_primitiva(p)
    v = parte_primitiva(q)

    # Paso 3: ciclo principal del algoritmo
    while v.degree() > 0:
        r = pseudoresto(u, v)
        u = v
        v = parte_primitiva(r)

    # Paso 4: resultado final
    return c * u if v == 0 else c * v

# Ejemplo
R.<x> = PolynomialRing(ZZ)
p = 54*x^3 - 54*x^2 + 84*x - 48
q = -12*x^3 - 28*x^2 + 72*x - 32

resultado = mcd_primitivo(p, q)
print(f"Resultado: {resultado}")

Resultado: -6*x + 4


In [2]:
def mcd_monico(p, q):
    """
    Algoritmo Euclidiano Mónico para polinomios con coeficientes racionales.
    Entrada: polinomios p y q donde grado(p) >= grado(q)
    Salida: mcd mónico(p, q) sobre los racionales Q
    """

    def hacer_monico(polinomio):
        """Convierte el polinomio en mónico dividiendo por su coeficiente líder"""
        if polinomio == 0:
            return polinomio
        return polinomio / polinomio.leading_coefficient()

    # Paso 1: hacer p y q mónicos
    u = hacer_monico(p)
    v = hacer_monico(q)

    # Paso 2: ciclo principal
    while v.degree() > 0:
        _, r = u.quo_rem(v)
        u = v
        v = hacer_monico(r) if r != 0 else r

    # Paso 3: resultado
    return u if v == 0 else 1

# Ejemplo
R.<x> = PolynomialRing(QQ)  # Polinomios con coeficientes racionales
p = 54*x^3 - 54*x^2 + 84*x - 48
q = -12*x^3 - 28*x^2 + 72*x - 32

resultado = mcd_monico(p, q)
print(f"Resultado: {resultado}")

Resultado: x - 2/3
